In [232]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras.backend as K
import keras.models as Sequential 
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import SGD,Adam
plt.style.use('classic')
%matplotlib inline
sns.set()
import knn_impute

In [233]:
train=pd.read_csv('dataset/train.csv')
test=pd.read_csv('dataset/test.csv')
submission=pd.read_csv('dataset/sample_submission.csv')


In [234]:
train.info()
train.head(70)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 17 columns):
id    552 non-null int64
A     544 non-null object
B     543 non-null float64
C     552 non-null float64
D     547 non-null object
E     547 non-null object
F     545 non-null object
G     545 non-null object
H     552 non-null float64
I     552 non-null object
J     552 non-null object
K     552 non-null int64
L     552 non-null object
M     552 non-null object
N     541 non-null float64
O     552 non-null int64
P     552 non-null int64
dtypes: float64(4), int64(4), object(9)
memory usage: 73.4+ KB


,id,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
0,1,b,18.42,10.415,y,p,aa,v,0.125,t,f,0,f,g,120.0,375,1
1,2,a,21.75,11.750,u,g,c,v,0.250,f,f,0,t,g,180.0,0,1
2,3,b,30.17,1.085,y,p,c,v,0.040,f,f,0,f,g,170.0,179,1
3,4,b,22.67,2.540,y,p,c,h,2.585,t,f,0,f,g,0.0,0,0
4,5,a,36.00,1.000,u,g,c,v,2.000,t,t,11,f,g,0.0,456,0
5,6,b,31.25,1.125,u,g,ff,ff,0.000,f,t,1,f,g,96.0,19,1
6,7,b,18.58,5.710,u,g,d,v,0.540,f,f,0,f,g,120.0,0,1
7,8,b,32.42,3.000,u,g,d,v,0.165,f,f,0,t,g,120.0,0,1
8,9,b,37.33,6.500,u,g,m,h,4.250,t,t,12,t,g,93.0,0,0
9,10,b,35.00,3.375,u,g,c,h,8.290,f,f,0,t,g,0.0,0,1


In [235]:
train.B=train.B.fillna(train.B.mean())

In [236]:
train.A=knn_impute.knn_impute(target=train['A'],
           attributes=train.A,aggregation_method="mode", 
           k_neighbors=3)
train.B=knn_impute.knn_impute(target=train['B'],
           attributes=train.B,aggregation_method="mean", 
           k_neighbors=4,
           numeric_distance='cityblock')
train.D=knn_impute.knn_impute(target=train['D'],
           attributes=train.D,aggregation_method="mode", 
           k_neighbors=3)
train.E=knn_impute.knn_impute(target=train['E'],
           attributes=train.E,aggregation_method="mode", 
           k_neighbors=3)
train.F=knn_impute.knn_impute(target=train['F'],
           attributes=train.F,aggregation_method="mode", 
           k_neighbors=3)
train.G=knn_impute.knn_impute(target=train['G'],
           attributes=train.G,aggregation_method="mode", 
           k_neighbors=3)

/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [237]:
train['N'].replace(np.nan, 0, inplace=True)

In [238]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 17 columns):
id    552 non-null int64
A     552 non-null object
B     552 non-null float64
C     552 non-null float64
D     552 non-null object
E     552 non-null object
F     552 non-null object
G     552 non-null object
H     552 non-null float64
I     552 non-null object
J     552 non-null object
K     552 non-null int64
L     552 non-null object
M     552 non-null object
N     552 non-null float64
O     552 non-null int64
P     552 non-null int64
dtypes: float64(4), int64(4), object(9)
memory usage: 73.4+ KB


In [239]:
train.corr()

,id,B,C,H,K,N,O,P
id,1.000000,-0.004999,-0.007231,-0.011982,0.045936,0.034160,-0.052236,-0.034468
B,-0.004999,1.000000,0.150936,0.405290,0.240397,-0.076134,0.014227,-0.163473
C,-0.007231,0.150936,1.000000,0.287374,0.301832,-0.205713,0.141069,-0.199471
H,-0.011982,0.405290,0.287374,1.000000,0.390157,-0.107721,0.050995,-0.325125
K,0.045936,0.240397,0.301832,0.390157,1.000000,-0.155181,0.066033,-0.438026
N,0.034160,-0.076134,-0.205713,-0.107721,-0.155181,1.000000,0.072993,0.109354
O,-0.052236,0.014227,0.141069,0.050995,0.066033,0.072993,1.000000,-0.172351
P,-0.034468,-0.163473,-0.199471,-0.325125,-0.438026,0.109354,-0.172351,1.000000


In [240]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train['B']=scaler.fit_transform(train['B'])
train['C']=scaler.fit_transform(train['C'])
train['H']=scaler.fit_transform(train['H'])
train['K']=scaler.fit_transform(train['K'])
train['N']=scaler.fit_transform(train['N'])
train['O']=scaler.fit_transform(train['O'])
train['P']=scaler.fit_transform(train['P'])

/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data eithe

In [241]:
train.head()

,id,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
0,1,b,0.070226,0.371964,y,p,aa,v,0.004386,t,f,0.000,f,g,0.103448,0.00375,1.0
1,2,a,0.120301,0.419643,u,g,c,v,0.008772,f,f,0.000,t,g,0.155172,0.00000,1.0
2,3,b,0.246917,0.038750,y,p,c,v,0.001404,f,f,0.000,f,g,0.146552,0.00179,1.0
3,4,b,0.134135,0.090714,y,p,c,h,0.090702,t,f,0.000,f,g,0.000000,0.00000,0.0
4,5,a,0.334586,0.035714,u,g,c,v,0.070175,t,t,0.275,f,g,0.000000,0.00456,0.0


In [242]:
train.A = pd.factorize(train.A)[0]
train.D = pd.factorize(train.D)[0]
train.E = pd.factorize(train.E)[0]
train.F = pd.factorize(train.F)[0]
train.G = pd.factorize(train.G)[0]
train.I = pd.factorize(train.I)[0]
train.J = pd.factorize(train.J)[0]
train.L = pd.factorize(train.L)[0]
train.M = pd.factorize(train.M)[0]

In [243]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 16 columns):
id    138 non-null int64
A     134 non-null object
B     135 non-null float64
C     138 non-null float64
D     137 non-null object
E     137 non-null object
F     136 non-null object
G     136 non-null object
H     138 non-null float64
I     138 non-null object
J     138 non-null object
K     138 non-null int64
L     138 non-null object
M     138 non-null object
N     136 non-null float64
O     138 non-null int64
dtypes: float64(4), int64(3), object(9)
memory usage: 17.3+ KB


In [244]:
test.info()
test.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 16 columns):
id    138 non-null int64
A     134 non-null object
B     135 non-null float64
C     138 non-null float64
D     137 non-null object
E     137 non-null object
F     136 non-null object
G     136 non-null object
H     138 non-null float64
I     138 non-null object
J     138 non-null object
K     138 non-null int64
L     138 non-null object
M     138 non-null object
N     136 non-null float64
O     138 non-null int64
dtypes: float64(4), int64(3), object(9)
memory usage: 17.3+ KB


,id,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
0,553,b,29.17,3.500,u,g,w,v,3.500,t,t,3,t,g,329.0,0
1,554,b,28.33,5.000,u,g,w,v,11.000,t,f,0,t,g,70.0,0
2,555,b,29.25,14.790,u,g,aa,v,5.040,t,t,5,t,g,168.0,0
3,556,a,44.17,6.665,u,g,q,v,7.375,t,t,3,t,g,0.0,0
4,557,b,18.08,5.500,u,g,k,v,0.500,t,f,0,f,g,80.0,0
5,558,a,45.00,4.585,u,g,k,h,1.000,f,f,0,t,s,240.0,0
6,559,b,31.92,4.460,u,g,cc,h,6.040,t,t,3,f,g,311.0,300
7,560,b,29.67,1.415,u,g,w,h,0.750,t,t,1,f,g,240.0,100
8,561,b,27.67,13.750,u,g,w,v,5.750,t,f,0,t,g,487.0,500
9,562,b,38.42,0.705,u,g,c,v,0.375,f,t,2,f,g,225.0,500


In [263]:
test.A=knn_impute.knn_impute(target=test['A'],
           attributes=test.A,aggregation_method="mode", 
           k_neighbors=2)
test.B=test.B.fillna(test.B.mean())
test.D=knn_impute.knn_impute(target=test['D'],
           attributes=test.D,aggregation_method="mode", 
           k_neighbors=3)
test.E=knn_impute.knn_impute(target=test['E'],
           attributes=test.E,aggregation_method="mode", 
           k_neighbors=3)
test.F=knn_impute.knn_impute(target=test['F'],
           attributes=test.F,aggregation_method="mode", 
           k_neighbors=3)
test.G=knn_impute.knn_impute(target=test['G'],
           attributes=test.G,aggregation_method="mode", 
           k_neighbors=3)

/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [254]:
test['N'].replace(np.nan, 0, inplace=True)

In [264]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 16 columns):
id    138 non-null int64
A     138 non-null object
B     138 non-null float64
C     138 non-null float64
D     138 non-null object
E     138 non-null object
F     138 non-null object
G     138 non-null object
H     138 non-null float64
I     138 non-null object
J     138 non-null object
K     138 non-null int64
L     138 non-null object
M     138 non-null object
N     138 non-null float64
O     138 non-null int64
dtypes: float64(4), int64(3), object(9)
memory usage: 17.3+ KB


In [267]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
test['B']=scaler.fit_transform(test['B'])
test['C']=scaler.fit_transform(test['C'])
test['H']=scaler.fit_transform(test['H'])
test['K']=scaler.fit_transform(test['K'])
test['N']=scaler.fit_transform(test['N'])
test['O']=scaler.fit_transform(test['O'])

/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data eithe

In [268]:
test.A = pd.factorize(test.A)[0]
test.D = pd.factorize(test.D)[0]
test.E = pd.factorize(test.E)[0]
test.F = pd.factorize(test.F)[0]
test.G = pd.factorize(test.G)[0]
test.I = pd.factorize(test.I)[0]
test.J = pd.factorize(test.J)[0]
test.L = pd.factorize(test.L)[0]
test.M = pd.factorize(test.M)[0]

In [256]:
target=train.pop('P')

In [257]:
X=train
X_test=test

In [258]:
from sklearn.ensemble import RandomForestClassifier
model_rfc = RandomForestClassifier(n_estimators=1000)
from sklearn.cross_validation import cross_val_score
cross_val_score(model_rfc, X, target, cv=3)


/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([ 0.87567568,  0.85869565,  0.83606557])

In [260]:
model_rfc.fit(X,target)
importances = model_rfc.feature_importances_
importances
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

for f in range(X.shape[1]):
    print(X.columns[indices[f]],end=', ')

Feature ranking:
1. feature 9 (0.301118)
2. feature 11 (0.094406)
3. feature 8 (0.089617)
4. feature 15 (0.080049)
5. feature 3 (0.069843)
6. feature 14 (0.064264)
7. feature 2 (0.061453)
8. feature 10 (0.058496)
9. feature 0 (0.052171)
10. feature 6 (0.051807)
11. feature 7 (0.023729)
12. feature 5 (0.013251)
13. feature 4 (0.011554)
14. feature 13 (0.010724)
15. feature 12 (0.009493)
16. feature 1 (0.008024)
I, K, H, O, C, N, B, J, id, F, G, E, D, M, L, A, 

In [261]:
submission.head()

,id,P
0,553,0
1,554,1
2,555,0
3,556,1
4,557,1


In [275]:
y_pred=model_rfc.predict(X_test)

In [276]:
y_pred=pd.DataFrame(y_pred)
y_pred['id']=test.id
y_pred.columns=['P','id']
y_pred=y_pred[['id','P']]

In [277]:
y_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 2 columns):
id    138 non-null int64
P     138 non-null float64
dtypes: float64(1), int64(1)
memory usage: 2.2 KB


In [278]:
y_pred.head()

,id,P
0,553,1.0
1,554,1.0
2,555,1.0
3,556,0.0
4,557,0.0


In [279]:
pd.DataFrame(y_pred).to_csv("submission.csv")